In [3]:
import mlagents
from mlagents_envs.environment import UnityEnvironment as UE
from mlagents_envs.envs.unity_parallel_env import UnityParallelEnv as UPZBE
from SAC_Distillation.DistilledSACAgent import DistilledSAC
from SAC_Distillation.Trajectories import SAC_ExperienceBuffer
from Hyperparameters import HYPERPARAMS as params
import numpy as np
import torch
import torch.nn as nn

In [ ]:
class UnityWrapper(nn.Module):
    def __init__(self, model):
        super(UnityWrapper, self).__init__()
        self.model = model

    def forward(self, x,y):
        mean,log_std = self.model(x,y)
        std = torch.exp(log_std)
        dist = torch.distributions.Normal(mean, std)
        action = dist.sample()
        action = torch.tanh(action)
        return action

In [5]:
agent = torch.load('SavedModels/SAC_Distilled_trained.pth')

C:\Users\rullo\AppData\Local\Temp\ipykernel_23264\1368402006.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  agent = torch.load('SavedModels/SAC_Distilled_trained.pth')


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Instantiate wrapper
wrapped_model = UnityWrapper(agent).eval()

# Example input (match your actual dimensions)
camera_obs_example = torch.randn(1, *(1, 84, 84), device=device)
vector_obs_example = torch.randn(1, *(24,), device=device)

# Export simplified ONNX
torch.onnx.export(
    wrapped_model,
    (camera_obs_example, vector_obs_example),
    "distilled_sac_model_unity.onnx",
    export_params=True,
    opset_version=12,
    input_names=['camera_obs', 'vector_obs'],
    output_names=['continuous_actions'],  # Unity expects exactly this naming
    dynamic_axes={
        'camera_obs': {0: 'batch_size'},
        'vector_obs': {0: 'batch_size'},
        'continuous_actions': {0: 'batch_size'}
    }
)


TypeError: UnityWrapper.forward() takes 2 positional arguments but 3 were given